In [4]:
config = {
    "model_type": "ShuttleNet",
    "output_folder_name": "./model/20230516-155621_ShuttleNet/",
    "data_folder": "./data/",
    "seed_value": 42,
    "max_ball_round": 70,
    "encode_length": 4,
    "batch_size": 32,
    "lr": 1e-4,
    "epochs": 150,
    "n_layers": 1,
    "shot_dim": 32,
    "area_num": 5,
    "area_dim": 32,
    "player_dim": 32,
    "encode_dim": 32,
    "num_directions": 1,
    "K": 5,
    "sample": 6,
    "gpu_num": 0,
    "seed_value": 42
}

In [6]:
import pandas as pd
import numpy as np
import ast
from badmintoncleaner import prepare_dataset

train_matches = pd.read_csv("./data/train.csv")

In [7]:
config = ast.literal_eval(open(f"./model/20230516-155621_ShuttleNet/config").readline())
config, train_dataset, train_dataloader, val_dataloader, test_dataloader, train_matches, val_matches, test_matches = prepare_dataset(config)

In [34]:
prediction = pd.read_csv('./model/20230516-155621_ShuttleNet/prediction.csv')
val_gt = pd.read_csv('./model/20230516-155621_ShuttleNet/val_gt.csv')
ground_truth = val_gt[['rally_id', 'ball_round', 'landing_x', 'landing_y', 'type']].groupby('rally_id').apply(lambda r: (r['ball_round'].values, r['landing_x'].values, r['landing_y'].values, r['type'].values))
group = prediction[['rally_id', 'sample_id', 'ball_round', 'landing_x', 'landing_y', 'short service', 'net shot', 'lob', 'clear', 'drop', 'push/rush', 'smash', 'defensive shot', 'drive', 'long service']].groupby('rally_id').apply(lambda r: (r['sample_id'].values, r['ball_round'].values, r['landing_x'].values, r['landing_y'].values, r['short service'].values, r['net shot'].values, r['lob'].values, r['clear'].values, r['drop'].values, r['push/rush'].values, r['smash'].values, r['defensive shot'].values, r['drive'].values, r['long service'].values))

type_list = ['short service', 'net shot', 'lob', 'clear', 'drop', 'push/rush', 'smash', 'defensive shot', 'drive', 'long service']

0

In [58]:
# Experiment by permutating the shot type and landing position
# rally_length: 4
# rally_id: 0, 1
# samples 1

# create df with thiese columns
#rally_id,ball_round,landing_x,landing_y,type,rally_length
type_list = ['short service', 'net shot', 'lob', 'clear', 'drop', 'push/rush', 'smash', 'defensive shot', 'drive', 'long service']
net_shot_min_x, net_shot_max_x, net_shot_min_y, net_shot_max_y = -0.5, 0.5, -0.5, 0.7

ground_truth = pd.DataFrame(columns=['rally_id', 'ball_round', 'landing_x', 'landing_y', 'type'])
prediction  = pd.DataFrame(columns=['rally_id', 'sample_id', 'ball_round', 'landing_x', 'landing_y', 'short service', 'net shot', 'lob', 'clear', 'drop', 'push/rush', 'smash', 'defensive shot', 'drive', 'long service'])

# shot type and landing area tally
landing_x, landing_y = [], []
landing_x_pred, landing_y_pred = [], []
ball_type = 'net shot'
for ball_round in range(10):
    if ball_round % 2 == 0:
        landing_x.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y.append(np.random.uniform(min(0,net_shot_max_y) , max(0,net_shot_max_y)))
        landing_x_pred.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y_pred.append(np.random.uniform(min(0,net_shot_max_y) , max(0,net_shot_max_y)))
    else:
        landing_x.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y.append(np.random.uniform(min(0,net_shot_min_y) , max(0,net_shot_min_y)))
        landing_x_pred.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y_pred.append(np.random.uniform(min(0,net_shot_min_y) , max(0,net_shot_min_y)))

biased_softmax = [1.0 if ball == ball_type else 0 for ball in type_list] 

new_val_gt = pd.DataFrame({
    'rally_id': [0]*10,
    'ball_round': np.arange(10) + 5,
    'landing_x': landing_x,
    'landing_y': landing_y,
    'type': [ball_type]*10
})

new_prediction = pd.DataFrame({
    'rally_id': [0]*10,
    'sample_id': [0]*10,
    'ball_round': np.arange(10) + 5,
    'landing_x': landing_x_pred,
    'landing_y': landing_y_pred,
    'short service': [biased_softmax[0]]*10,
    'net shot': [biased_softmax[1]]*10,
    'lob': [biased_softmax[2]]*10,
    'clear': [biased_softmax[3]]*10,
    'drop': [biased_softmax[4]]*10,
    'push/rush': [biased_softmax[5]]*10,
    'smash': [biased_softmax[6]]*10,
    'defensive shot': [biased_softmax[7]]*10,
    'drive': [biased_softmax[8]]*10,
    'long service': [biased_softmax[9]]*10
})

ground_truth = pd.concat([ground_truth, new_val_gt], ignore_index=True)
prediction = pd.concat([prediction, new_prediction], ignore_index=True)

# landing area not tally but shot type tally
landing_x, landing_y = [],[]
landing_x_pred, landing_y_pred = [], []
ball_type = 'net shot'
for ball_round in range(10):
    if ball_round % 2 == 0:
        landing_x.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y.append(np.random.uniform(min(0,net_shot_max_y) , max(0,net_shot_max_y)))
        landing_x_pred.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y_pred.append(np.random.uniform(min(net_shot_max_y, 2.0) , max(net_shot_max_y, 2.0)))
    else:
        landing_x.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y.append(np.random.uniform(min(0,net_shot_min_y) , max(0,net_shot_min_y)))
        landing_x_pred.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y_pred.append(np.random.uniform(min(net_shot_min_y, -2.0) , max(net_shot_min_y, -2.0)))

biased_softmax = [1.0 if ball == ball_type else 0 for ball in type_list] 

new_val_gt = pd.DataFrame({
    'rally_id': [1]*10,
    'ball_round': np.arange(10) + 5,
    'landing_x': landing_x,
    'landing_y': landing_y,
    'type': ['net shot']*10
})

new_prediction = pd.DataFrame({
    'rally_id': [1]*10,
    'sample_id': [0]*10,
    'ball_round': np.arange(10) + 5,
    'landing_x': landing_x_pred,
    'landing_y': landing_y_pred,
    'short service': [biased_softmax[0]]*10,
    'net shot': [biased_softmax[1]]*10,
    'lob': [biased_softmax[2]]*10,
    'clear': [biased_softmax[3]]*10,
    'drop': [biased_softmax[4]]*10,
    'push/rush': [biased_softmax[5]]*10,
    'smash': [biased_softmax[6]]*10,
    'defensive shot': [biased_softmax[7]]*10,
    'drive': [biased_softmax[8]]*10,
    'long service': [biased_softmax[9]]*10
})

ground_truth = pd.concat([ground_truth, new_val_gt], ignore_index=True)
prediction = pd.concat([prediction, new_prediction], ignore_index=True)

# landing area tally but shot type not tally
landing_x, landing_y = [],[]
landing_x_pred, landing_y_pred = [], []
ball_type = 'lob'
for ball_round in range(10):
    if ball_round % 2 == 0:
        landing_x.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y.append(np.random.uniform(min(0,net_shot_max_y) , max(0,net_shot_max_y)))
        landing_x_pred.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y_pred.append(np.random.uniform(min(0,net_shot_max_y) , max(0,net_shot_max_y)))
    else:
        landing_x.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y.append(np.random.uniform(min(0,net_shot_min_y) , max(0,net_shot_min_y)))
        landing_x_pred.append(np.random.uniform(net_shot_min_x, net_shot_max_x))
        landing_y_pred.append(np.random.uniform(min(0,net_shot_min_y) , max(0,net_shot_min_y)))

biased_softmax = [0.5 if ball in ['lob', 'smash'] else 0 for ball in type_list] 

new_val_gt = pd.DataFrame({
    'rally_id': [2]*10,
    'ball_round': np.arange(10) + 5,
    'landing_x': landing_x,
    'landing_y': landing_y,
    'type': ['net shot']*10
})

new_prediction = pd.DataFrame({
    'rally_id': [2]*10,
    'sample_id': [0]*10,
    'ball_round': np.arange(10) + 5,
    'landing_x': landing_x_pred,
    'landing_y': landing_y_pred,
    'short service': [biased_softmax[0]]*10,
    'net shot': [biased_softmax[1]]*10,
    'lob': [biased_softmax[2]]*10,
    'clear': [biased_softmax[3]]*10,
    'drop': [biased_softmax[4]]*10,
    'push/rush': [biased_softmax[5]]*10,
    'smash': [biased_softmax[6]]*10,
    'defensive shot': [biased_softmax[7]]*10,
    'drive': [biased_softmax[8]]*10,
    'long service': [biased_softmax[9]]*10
})

ground_truth = pd.concat([ground_truth, new_val_gt], ignore_index=True)
prediction = pd.concat([prediction, new_prediction], ignore_index=True)

In [60]:
ground_truth.to_csv('./model/mangcai/val_gt.csv', index=False)
prediction.to_csv('./model/mangcai/prediction.csv', index=False)